# Step 2: EfficientNet-B0 Semantic Baseline
## Proving the Pretrained Backbone Hypothesis on Kaggle T4

**Concept proof conclusion:** No handcrafted feature separates real from fake on FF++ c23.  
The signal is semantic — only accessible through ImageNet-pretrained features.

**This notebook:** Train EfficientNet-B0 (pretrained ImageNet) on FF++ and measure  
cross-dataset AUC on Celeb-DF. This gives us the floor V8.0 must beat.

### Key Fixes in This Version
- ✅ **Parallel frame extraction** via `ThreadPoolExecutor` (8 threads) — ~10x faster than single-threaded sequential scan
- ✅ **Keyframe-anchored seek** — one `cap.set()` per video to first target, then linear read — avoids full-file buffering on network FS
- ✅ **All frames cached to RAM** before training — zero filesystem I/O in DataLoader
- ✅ **Batch-level tqdm** inside train/eval loops — training never silently hangs
- ✅ **Video validation** step before training to catch corrupt files early
- ✅ **Flush after every epoch print** so output appears immediately in Kaggle
- ℹ️  **GPU is idle during extraction** — that is correct and expected; GPU activates in Section 5

### Expected Timeline on Kaggle T4
| Phase | Time |
|-------|------|
| Frame extraction (8 threads) | ~3-5 min |
| Training 20 epochs | ~50-70 min |
| Evaluation on FF++ val | ~3 min |
| Evaluation on Celeb-DF | ~3 min |
| **Total** | **~60-80 min** |

### Success Criteria
- **> 75% Celeb-DF AUC** → semantic signal is strong → V8.0 temporal module will push higher  
- **60-75%** → harder problem → need multi-method training  
- **< 60%** → something wrong with training setup → debug first

## Section 1 — Setup & Imports

In [ ]:
import os, sys, json, random, time, warnings
from pathlib import Path
from typing import Optional, List, Tuple
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score, roc_curve, confusion_matrix
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from tqdm.auto import tqdm
warnings.filterwarnings('ignore')

# ── CRITICAL: Disable OpenCV threading BEFORE any cv2 import ──────────────────
# Must happen before cv2 is imported to prevent deadlocks with PyTorch DataLoader
import cv2
cv2.setNumThreads(0)
cv2.ocl.setUseOpenCL(False)

# ── Reproducibility ────────────────────────────────────────────────────────────
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

# ── Device ─────────────────────────────────────────────────────────────────────
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Device: {DEVICE}')
if torch.cuda.is_available():
    print(f'GPU: {torch.cuda.get_device_name(0)}')
    print(f'VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB')

# ── Paths ──────────────────────────────────────────────────────────────────────
OUTPUT_DIR = Path('/kaggle/working/baseline')
CKPT_DIR   = OUTPUT_DIR / 'checkpoints'
PLOTS_DIR  = OUTPUT_DIR / 'plots'
for d in [OUTPUT_DIR, CKPT_DIR, PLOTS_DIR]:
    d.mkdir(parents=True, exist_ok=True)

print(f'Outputs → {OUTPUT_DIR}')
print(f'tqdm version: {tqdm.__module__}')

In [ ]:
# ── Training config ────────────────────────────────────────────────────────────
CFG = {
    # Data
    'img_size':        224,
    'n_frames':        4,       # frames extracted per video
    'n_train_real':    300,
    'n_train_fake':    300,
    'n_val':           100,

    # Training
    'epochs':          20,
    'batch_size':      32,
    'lr':              1e-4,
    'weight_decay':    1e-4,
    'warmup_epochs':   2,

    # Model
    'dropout':         0.3,
    'label_smoothing': 0.0,

    # DataLoader — keep 0 workers; cv2+fork = deadlock
    'num_workers':     0,
    'pin_memory':      False,

    # Frame extraction: read sequentially instead of random seek
    # This is the main fix for Kaggle /kaggle/input network-mounted FS
    'use_sequential_read': True,
}

print('Config:')
for k, v in CFG.items():
    print(f'  {k:25s}: {v}')

## Section 2 — Dataset Paths

In [ ]:
KAGGLE_INPUT = Path('/kaggle/input')

def locate_ff_root(base):
    known = base / 'datasets' / 'xdxd003' / 'ff-c23' / 'FaceForensics++_C23'
    if known.exists(): return known
    for d in sorted(base.rglob('*')):
        if d.is_dir():
            if sum(1 for m in ['Deepfakes', 'Face2Face', 'FaceSwap']
                   if (d / m).exists()) >= 2:
                return d
    return None

def locate_celeb_root(base):
    known = base / 'datasets' / 'reubensuju' / 'celeb-df-v2'
    if known.exists(): return known
    for d in sorted(base.rglob('*')):
        if d.is_dir() and (d / 'Celeb-real').exists():
            return d
    return None

FF_ROOT    = locate_ff_root(KAGGLE_INPUT)
CELEB_ROOT = locate_celeb_root(KAGGLE_INPUT)
print(f'FF++    : {FF_ROOT}')
print(f'Celeb-DF: {CELEB_ROOT}')

assert FF_ROOT is not None,    'ERROR: FF++ root not found — check dataset mount'
assert CELEB_ROOT is not None, 'ERROR: Celeb-DF root not found — check dataset mount'

# ── Collect video paths ────────────────────────────────────────────────────────
def get_ff_videos(root):
    real = sorted(root.rglob('original*/*.mp4'))
    if not real:
        real = sorted(p for p in root.rglob('*.mp4') if 'original' in str(p).lower())
    fakes = sorted((root / 'Deepfakes').glob('*.mp4')) if (root / 'Deepfakes').exists() else []
    return real, fakes

def get_celeb_videos(root):
    real = (sorted((root / 'Celeb-real').glob('*.mp4')) +
            sorted((root / 'YouTube-real').glob('*.mp4')))
    fake = sorted((root / 'Celeb-synthesis').glob('*.mp4'))
    return real, fake

FF_REAL, FF_FAKE   = get_ff_videos(FF_ROOT)
CDF_REAL, CDF_FAKE = get_celeb_videos(CELEB_ROOT)

print(f'FF++ real: {len(FF_REAL)}, fake(Deepfakes): {len(FF_FAKE)}')
print(f'Celeb-DF real: {len(CDF_REAL)}, fake: {len(CDF_FAKE)}')
assert len(FF_REAL) > 0 and len(FF_FAKE) > 0,   'ERROR: No FF++ videos found'
assert len(CDF_REAL) > 0 and len(CDF_FAKE) > 0, 'ERROR: No Celeb-DF videos found'

# ── Splits ─────────────────────────────────────────────────────────────────────
rng = random.Random(SEED)

def make_split(real_list, fake_list, n_real, n_fake, n_val_each=50):
    real_s = rng.sample(real_list, min(n_real + n_val_each, len(real_list)))
    fake_s = rng.sample(fake_list, min(n_fake + n_val_each, len(fake_list)))
    train = ([(p, 0) for p in real_s[:n_real]] +
             [(p, 1) for p in fake_s[:n_fake]])
    val   = ([(p, 0) for p in real_s[n_real:n_real + n_val_each]] +
             [(p, 1) for p in fake_s[n_fake:n_fake + n_val_each]])
    rng.shuffle(train)
    rng.shuffle(val)
    return train, val

TRAIN_DATA, VAL_DATA = make_split(
    FF_REAL, FF_FAKE,
    CFG['n_train_real'], CFG['n_train_fake'],
    n_val_each=50
)

n_cdf = min(200, len(CDF_REAL), len(CDF_FAKE))
CDF_TEST = ([(p, 0) for p in rng.sample(CDF_REAL, n_cdf)] +
            [(p, 1) for p in rng.sample(CDF_FAKE, n_cdf)])

print(f'\nSplits:')
print(f'  Train : {len(TRAIN_DATA)} videos')
print(f'  Val   : {len(VAL_DATA)} videos')
print(f'  CDF   : {len(CDF_TEST)} videos (cross-dataset test)')

## Section 2b — Video Validation

Probe the first N videos to catch corrupt files or mount issues **before** training begins.  
This also confirms that frame seeking actually works on the Kaggle filesystem.

In [ ]:
# ── Video validation — runs quickly, catches problems before training ──────────
def probe_video(path, n_frames=CFG['n_frames']):
    """
    Returns (ok: bool, frame_count: int, error_msg: str).
    Tries a SEQUENTIAL read strategy: scan forward frame-by-frame.
    This is more reliable than cap.set(POS_FRAMES) on network filesystems.
    """
    try:
        cap = cv2.VideoCapture(str(path))
        if not cap.isOpened():
            return False, 0, 'cannot open'
        total = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        # Try reading frame 0 to confirm file is accessible
        ret, frame = cap.read()
        cap.release()
        if not ret or frame is None:
            return False, total, 'read frame 0 failed'
        return True, total, ''
    except Exception as e:
        return False, 0, str(e)

print('Validating video files (sampling first 20 of each split)...')
bad_videos = []
all_to_check = TRAIN_DATA[:20] + VAL_DATA[:10] + CDF_TEST[:10]

for path, label in tqdm(all_to_check, desc='Probing'):
    ok, n, err = probe_video(path)
    if not ok:
        bad_videos.append((path, err))
        print(f'  ❌ {path.name}: {err}')

if bad_videos:
    print(f'\n⚠️  {len(bad_videos)} bad video(s) found. They will return black frames during training.')
else:
    print(f'\n✅ All {len(all_to_check)} sampled videos OK — dataset is readable.')

## Section 3 — Fast Parallel Frame Extraction & Dataset

### Why GPU shows 0% here — and that's fine
Frame extraction is a **CPU + disk I/O** task. OpenCV has no GPU decode path in this environment.
The GPU will jump to 80-100% utilization in Section 5 once the training loop starts.

### Speed problem & fix
Single-threaded sequential scan at 3.8s/video × 600 videos = **~38 min** — unacceptable.  
Fix: **`ThreadPoolExecutor` with 8 threads** runs 8 videos simultaneously.  
Threads are safe here (unlike `multiprocessing`) because:
- `cv2` releases the GIL during decode → real parallelism despite Python's GIL
- No `fork()` = no cv2 deadlock
- Kaggle T4 nodes have 2 CPUs with hyperthreading → 8 threads is the sweet spot

### Seek strategy
Instead of a full sequential scan from frame 0, we do **one `cap.set()` seek to the first target frame** then read linearly from there. This skips buffering the first portion of the file while staying reliable on the network FS.

In [ ]:
# ── Frame extraction helper ────────────────────────────────────────────────────
# Strategy: use cap.set(POS_FRAMES) for the FIRST frame only (keyframe seek),
# then read subsequent target frames sequentially from that point.
# This avoids full-file buffering while still being much faster than pure
# sequential scan. Falls back to a full sequential scan if seek fails.

from concurrent.futures import ThreadPoolExecutor, as_completed

N_EXTRACT_WORKERS = 8   # Thread-based parallelism — safe with cv2 (no fork)

def _postprocess_frame(frame_bgr, img_size):
    """BGR numpy → center-cropped RGB resized uint8."""
    frame_rgb = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2RGB)
    h, w = frame_rgb.shape[:2]
    frame_rgb = frame_rgb[int(h * 0.05):int(h * 0.95),
                          int(w * 0.10):int(w * 0.90)]
    return cv2.resize(frame_rgb, (img_size, img_size))


def extract_frames_from_video(video_path, n_frames, img_size):
    """
    Fast frame extraction using keyframe-anchored seek.

    Algorithm:
      1. Open video, read total frame count.
      2. Compute N evenly-spaced target indices.
      3. Seek to the FIRST target (one seek per video max).
      4. Read frames sequentially from that anchor point,
         collecting each target as we pass it.
      5. On any failure, fall back to reading frame 0 × n_frames.

    This is ~5-10x faster than the pure sequential scan on Kaggle's
    network FS because we skip the first portion of the video in one
    seek call and then read linearly from there.
    """
    black = np.zeros((img_size, img_size, 3), dtype=np.uint8)

    try:
        cap = cv2.VideoCapture(str(video_path))
        if not cap.isOpened():
            cap.release()
            return [black.copy() for _ in range(n_frames)]

        total = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        if total < 1:
            # Unknown length: just read up to 300 frames to find content
            total = 300

        targets = sorted(set(np.linspace(0, total - 1, n_frames, dtype=int).tolist()))
        collected = {}

        # Seek to just before the first target frame
        first_target = targets[0]
        if first_target > 0:
            cap.set(cv2.CAP_PROP_POS_FRAMES, first_target)
            actual_pos = int(cap.get(cv2.CAP_PROP_POS_FRAMES))
            # If seek overshot, reset to 0 and scan from start
            if actual_pos > first_target:
                cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
                current = 0
            else:
                current = actual_pos
        else:
            current = 0

        target_set = set(targets)
        last_target = targets[-1]

        while current <= last_target + 5:  # +5 slack for seek imprecision
            ret, frame = cap.read()
            if not ret:
                break
            if current in target_set:
                collected[current] = _postprocess_frame(frame, img_size)
                if len(collected) == n_frames:
                    break
            current += 1

        cap.release()

        # If seek was imprecise, some targets may be off-by-1; try nearest
        frames = []
        for t in targets:
            if t in collected:
                frames.append(collected[t])
            else:
                # Try ±2 neighbours
                found = None
                for delta in [1, -1, 2, -2]:
                    if (t + delta) in collected:
                        found = collected[t + delta]
                        break
                frames.append(found if found is not None else black.copy())

        return frames[:n_frames]

    except Exception:
        return [black.copy() for _ in range(n_frames)]


def _extract_one(args):
    """Worker function for ThreadPoolExecutor."""
    path, label, n_frames, img_size = args
    frames = extract_frames_from_video(str(path), n_frames, img_size)
    return [(f, label) for f in frames]


# ── Transforms ────────────────────────────────────────────────────────────────
IMAGENET_MEAN = [0.485, 0.456, 0.406]
IMAGENET_STD  = [0.229, 0.224, 0.225]

train_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.1, hue=0.05),
    transforms.RandomGrayscale(p=0.05),
    transforms.ToTensor(),
    transforms.Normalize(IMAGENET_MEAN, IMAGENET_STD),
])

val_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.ToTensor(),
    transforms.Normalize(IMAGENET_MEAN, IMAGENET_STD),
])


class DeepfakeFrameDataset(Dataset):
    """
    Pre-extracts all frames at __init__ time using a thread pool,
    then caches them in RAM as uint8 numpy arrays.
    __getitem__ only touches RAM — zero filesystem I/O during training.

    Why threads and not processes?
    - cv2 + fork (multiprocessing) = deadlock on Kaggle
    - Threads share the GIL but cv2 decode releases it → real parallelism
    - ThreadPoolExecutor is safe, no zombie processes, clean shutdown
    """
    def __init__(self, video_label_pairs: list, transform,
                 n_frames: int = CFG['n_frames'],
                 img_size: int = CFG['img_size'],
                 augment: bool = True,
                 desc: str = 'Extracting',
                 n_workers: int = N_EXTRACT_WORKERS):
        self.transform = transform
        self.augment   = augment
        self.items     = []  # list of (frame_np_uint8, label_int)

        n_total = len(video_label_pairs) * n_frames
        print(f'{desc}: {len(video_label_pairs)} videos × {n_frames} frames '
              f'= {n_total} items  [{n_workers} threads]')

        args_list = [(p, lbl, n_frames, img_size) for p, lbl in video_label_pairs]

        # Preserve original order: submit all, collect in submission order
        results_ordered = [None] * len(args_list)
        with ThreadPoolExecutor(max_workers=n_workers) as pool:
            future_to_idx = {pool.submit(_extract_one, a): i
                             for i, a in enumerate(args_list)}
            pbar = tqdm(as_completed(future_to_idx),
                        total=len(args_list), desc=desc, leave=True)
            for future in pbar:
                idx = future_to_idx[future]
                try:
                    results_ordered[idx] = future.result()
                except Exception as e:
                    black = np.zeros((img_size, img_size, 3), dtype=np.uint8)
                    lbl   = video_label_pairs[idx][1]
                    results_ordered[idx] = [(black, lbl)] * n_frames

        for video_frames in results_ordered:
            self.items.extend(video_frames)

        print(f'  → {len(self.items)} frames cached in RAM  '
              f'({len(self.items) * img_size * img_size * 3 / 1e6:.0f} MB)')

    def __len__(self):
        return len(self.items)

    def __getitem__(self, idx):
        frame, label = self.items[idx]
        return self.transform(frame), torch.tensor(label, dtype=torch.long)


# ── Time the extraction ────────────────────────────────────────────────────────
print('Building datasets (parallel frame extraction)...')
print(f'Using {N_EXTRACT_WORKERS} threads — GPU stays idle here, that is normal.')
print('─' * 65)

t0 = time.time()
train_ds = DeepfakeFrameDataset(TRAIN_DATA, train_transform, augment=True,  desc='Train')
print(f'  Train done: {(time.time()-t0):.1f}s')

t1 = time.time()
val_ds   = DeepfakeFrameDataset(VAL_DATA,   val_transform,   augment=False, desc='Val')
print(f'  Val done:   {(time.time()-t1):.1f}s')

t2 = time.time()
cdf_ds   = DeepfakeFrameDataset(CDF_TEST,   val_transform,   augment=False, desc='CDF')
print(f'  CDF done:   {(time.time()-t2):.1f}s')

print(f'\nTotal extraction time: {(time.time()-t0)/60:.1f} min  '
      f'(was ~38 min sequential, now ~3-5 min parallel)')

# ── DataLoaders ────────────────────────────────────────────────────────────────
train_loader = DataLoader(train_ds, batch_size=CFG['batch_size'],
                          shuffle=True,  num_workers=0, pin_memory=False)
val_loader   = DataLoader(val_ds,   batch_size=CFG['batch_size'],
                          shuffle=False, num_workers=0, pin_memory=False)
cdf_loader   = DataLoader(cdf_ds,   batch_size=CFG['batch_size'],
                          shuffle=False, num_workers=0, pin_memory=False)

print(f'\nDataLoader summary:')
print(f'  Train : {len(train_ds):6d} frames,  {len(train_loader):4d} batches')
print(f'  Val   : {len(val_ds):6d} frames,  {len(val_loader):4d} batches')
print(f'  CDF   : {len(cdf_ds):6d} frames,  {len(cdf_loader):4d} batches')

# ── Sanity check — one batch ──────────────────────────────────────────────────
print('\nSanity check: loading one batch from RAM...')
x_b, y_b = next(iter(train_loader))
print(f'  x shape : {x_b.shape}, dtype: {x_b.dtype}')
print(f'  y shape : {y_b.shape}, label balance: {y_b.sum().item()}/{len(y_b)}')
print('✅ DataLoader OK — GPU will activate in Section 5 during model forward pass.')

## Section 4 — Model: EfficientNet-B0

EfficientNet-B0 pretrained on ImageNet-1K.  
We replace the classifier head with a binary deepfake detector.

**Why B0 and not B4?**  
B0 trains in ~60 min on T4. B4 would take 3-4 hours and we just need  
a baseline number — not maximum performance. We can upgrade to B4 on RunPod.

In [ ]:
class DeepfakeDetector(nn.Module):
    """
    EfficientNet-B0 pretrained on ImageNet + custom classification head.
    The backbone provides semantic face features that survive compression.
    """
    def __init__(self, dropout: float = CFG['dropout']):
        super().__init__()
        self.backbone = models.efficientnet_b0(
            weights=models.EfficientNet_B0_Weights.IMAGENET1K_V1
        )
        in_features = self.backbone.classifier[1].in_features  # 1280

        self.backbone.classifier = nn.Sequential(
            nn.Dropout(p=dropout),
            nn.Linear(in_features, 256),
            nn.ReLU(inplace=True),
            nn.Dropout(p=dropout * 0.5),
            nn.Linear(256, 2),  # real=0, fake=1
        )

        # Track param groups for differential LRs
        self.backbone_params = list(self.backbone.features.parameters())
        self.head_params     = list(self.backbone.classifier.parameters())

        print(f'Backbone params : {sum(p.numel() for p in self.backbone_params):,}')
        print(f'Head params     : {sum(p.numel() for p in self.head_params):,}')
        print(f'Total params    : {sum(p.numel() for p in self.parameters()):,}')

    def forward(self, x):
        return self.backbone(x)

    def get_param_groups(self, base_lr: float):
        """Differential LRs: backbone 10× lower than head."""
        return [
            {'params': self.backbone_params, 'lr': base_lr / 10},  # 1e-5
            {'params': self.head_params,     'lr': base_lr},        # 1e-4
        ]


model = DeepfakeDetector().to(DEVICE)
print(f'\nModel on: {DEVICE}')

# Verify forward pass
with torch.no_grad():
    _out = model(torch.randn(2, 3, 224, 224).to(DEVICE))
    print(f'Forward pass: (2,3,224,224) → {_out.shape}  ✅')

## Section 5 — Training Loop

In [ ]:
# ── Loss, optimizer, scheduler ────────────────────────────────────────────────
criterion = nn.CrossEntropyLoss(label_smoothing=CFG['label_smoothing'])

optimizer = torch.optim.AdamW(
    model.get_param_groups(CFG['lr']),
    weight_decay=CFG['weight_decay']
)

def get_lr_lambda(epoch):
    if epoch < CFG['warmup_epochs']:
        return (epoch + 1) / CFG['warmup_epochs']
    progress = (epoch - CFG['warmup_epochs']) / max(1, CFG['epochs'] - CFG['warmup_epochs'])
    return 0.5 * (1 + np.cos(np.pi * progress))

scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, get_lr_lambda)

print('Optimizer : AdamW with differential LRs')
print(f'  Backbone LR : {CFG["lr"]/10:.2e}')
print(f'  Head LR     : {CFG["lr"]:.2e}')
print(f'Scheduler : Linear warmup ({CFG["warmup_epochs"]} epochs) + Cosine decay')

In [ ]:
# ── Evaluation function ───────────────────────────────────────────────────────
def evaluate(model, loader, device, desc='Eval'):
    model.eval()
    all_labels, all_probs, all_preds = [], [], []
    total_loss = 0.0
    n_batches  = 0

    with torch.no_grad():
        pbar = tqdm(loader, desc=desc, leave=False, dynamic_ncols=True)
        for x, y in pbar:
            x, y   = x.to(device), y.to(device)
            logits  = model(x)
            loss    = criterion(logits, y)
            probs   = F.softmax(logits, dim=1)[:, 1]

            total_loss += loss.item()
            n_batches  += 1
            all_labels.extend(y.cpu().numpy())
            all_probs.extend(probs.cpu().numpy())
            all_preds.extend(logits.argmax(1).cpu().numpy())

            pbar.set_postfix(loss=f'{loss.item():.4f}')

    labels = np.array(all_labels)
    probs  = np.array(all_probs)
    preds  = np.array(all_preds)

    auc  = roc_auc_score(labels, probs) if len(np.unique(labels)) > 1 else 0.5
    acc  = (preds == labels).mean()
    loss = total_loss / max(n_batches, 1)
    return {'auc': auc, 'acc': acc, 'loss': loss, 'labels': labels, 'probs': probs}


# ── Training function ─────────────────────────────────────────────────────────
def train_epoch(model, loader, optimizer, device):
    model.train()
    total_loss = 0.0
    correct    = 0
    total      = 0

    pbar = tqdm(loader, desc='Train', leave=False, dynamic_ncols=True)
    for batch_idx, (x, y) in enumerate(pbar):
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        logits = model(x)
        loss   = criterion(logits, y)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()

        total_loss += loss.item()
        correct    += (logits.argmax(1) == y).sum().item()
        total      += y.size(0)

        pbar.set_postfix(
            loss=f'{loss.item():.4f}',
            acc=f'{correct/total:.3f}',
            batch=f'{batch_idx+1}/{len(loader)}'
        )

    return total_loss / len(loader), correct / total


print('✅ Training functions ready — tqdm progress bars will show batch-level progress.')

In [ ]:
# ── Main training loop ────────────────────────────────────────────────────────
history = {'train_loss': [], 'train_acc': [], 'val_loss': [],
           'val_auc': [], 'val_acc': [], 'lr': []}

best_val_auc = 0.0
best_epoch   = 0
start_time   = time.time()

HDR = f"{'Ep':>3} {'TrLoss':>8} {'TrAcc':>7} {'VaLoss':>8} {'VaAUC':>7} {'VaAcc':>7} {'HeadLR':>9} {'Min':>5}"
print('=' * len(HDR))
print(HDR)
print('=' * len(HDR))
sys.stdout.flush()

epoch_pbar = tqdm(range(CFG['epochs']), desc='Epochs', position=0)

for epoch in epoch_pbar:
    t0 = time.time()

    tr_loss, tr_acc = train_epoch(model, train_loader, optimizer, DEVICE)
    val_metrics     = evaluate(model, val_loader, DEVICE, desc=f'Val E{epoch+1}')

    scheduler.step()
    current_lr = optimizer.param_groups[1]['lr']

    history['train_loss'].append(tr_loss)
    history['train_acc'].append(tr_acc)
    history['val_loss'].append(val_metrics['loss'])
    history['val_auc'].append(val_metrics['auc'])
    history['val_acc'].append(val_metrics['acc'])
    history['lr'].append(current_lr)

    elapsed_min = (time.time() - t0) / 60
    is_best = val_metrics['auc'] > best_val_auc
    flag    = ' ←' if is_best else ''

    row = (f"{epoch+1:>3} {tr_loss:>8.4f} {tr_acc:>7.3f} "
           f"{val_metrics['loss']:>8.4f} {val_metrics['auc']:>7.4f} "
           f"{val_metrics['acc']:>7.3f} {current_lr:>9.2e} "
           f"{elapsed_min:>4.1f}m{flag}")
    print(row)
    sys.stdout.flush()  # Force immediate output in Kaggle

    epoch_pbar.set_postfix(val_auc=f"{val_metrics['auc']:.4f}", best=f"{best_val_auc:.4f}")

    if is_best:
        best_val_auc = val_metrics['auc']
        best_epoch   = epoch + 1
        torch.save({
            'epoch':       epoch,
            'model_state': model.state_dict(),
            'val_auc':     best_val_auc,
            'cfg':         CFG,
        }, CKPT_DIR / 'best.pth')

total_time = time.time() - start_time
print('=' * len(HDR))
print(f'\nBest Val AUC : {best_val_auc:.4f} at epoch {best_epoch}')
print(f'Total time   : {total_time/60:.1f} min')
sys.stdout.flush()

## Section 6 — Evaluation

In [ ]:
# ── Load best checkpoint ───────────────────────────────────────────────────────
# weights_only=False required because PyTorch 2.6 changed the default to True,
# and older checkpoints saved with numpy scalars fail the safe unpickler.
# Safe here: we wrote this checkpoint ourselves in the cell above.
ckpt = torch.load(CKPT_DIR / 'best.pth', map_location=DEVICE, weights_only=False)
model.load_state_dict(ckpt['model_state'])
print(f"Loaded best model from epoch {ckpt['epoch']+1}  (val AUC={ckpt['val_auc']:.4f})")

# ── Evaluate on FF++ val ──────────────────────────────────────────────────────
print('\nEvaluating on FF++ validation set...')
ff_metrics = evaluate(model, val_loader, DEVICE, desc='FF++ Val')
print(f"  AUC={ff_metrics['auc']:.4f}  Acc={ff_metrics['acc']:.4f}")

# ── Evaluate on Celeb-DF (cross-dataset) ──────────────────────────────────────
print('\nEvaluating on Celeb-DF (cross-dataset test)...')
cdf_metrics = evaluate(model, cdf_loader, DEVICE, desc='Celeb-DF')
print(f"  AUC={cdf_metrics['auc']:.4f}  Acc={cdf_metrics['acc']:.4f}")

gap = ff_metrics['auc'] - cdf_metrics['auc']
print('\n' + '=' * 52)
print('BASELINE RESULTS SUMMARY')
print('=' * 52)
print(f"  FF++ Val  AUC : {ff_metrics['auc']:.4f}")
print(f"  Celeb-DF  AUC : {cdf_metrics['auc']:.4f}  ← cross-dataset generalization")
print(f"  Gap           : {gap:.4f}")
print('=' * 52)

if cdf_metrics['auc'] >= 0.75:
    print('🟢 STRONG — Semantic signal confirmed. V8.0 temporal module will push higher.')
elif cdf_metrics['auc'] >= 0.60:
    print('🟡 MODERATE — Signal exists but weak. Train on all FF++ methods next.')
else:
    print('🔴 WEAK — Check training setup. Consider longer training or B4 backbone.')

In [ ]:
# ── Training curves ───────────────────────────────────────────────────────────
fig, axes = plt.subplots(1, 3, figsize=(18, 5))
fig.suptitle('EfficientNet-B0 Baseline — Training Curves', fontsize=14, fontweight='bold')

x = range(1, len(history['train_loss']) + 1)

axes[0].plot(x, history['train_loss'], label='Train', color='#3498db', linewidth=2)
axes[0].plot(x, history['val_loss'],   label='Val',   color='#e74c3c', linewidth=2)
axes[0].set_title('Loss'); axes[0].set_xlabel('Epoch')
axes[0].legend(); axes[0].grid(True, alpha=0.3)

axes[1].plot(x, history['val_auc'], color='#2ecc71', linewidth=2.5, label='Val AUC')
axes[1].axhline(best_val_auc,        color='gray',    linestyle='--', alpha=0.7,
                label=f'Best={best_val_auc:.4f}')
axes[1].axhline(cdf_metrics['auc'],  color='#e74c3c', linestyle='--', alpha=0.7,
                label=f'CDF={cdf_metrics["auc"]:.4f}')
axes[1].set_title('Val AUC'); axes[1].set_xlabel('Epoch')
axes[1].legend(); axes[1].grid(True, alpha=0.3); axes[1].set_ylim(0.4, 1.0)

axes[2].plot(x, history['lr'], color='#9b59b6', linewidth=2)
axes[2].set_title('Head Learning Rate'); axes[2].set_xlabel('Epoch')
axes[2].set_yscale('log'); axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(PLOTS_DIR / 'training_curves.png', dpi=150, bbox_inches='tight')
plt.show()
print('✅ Saved training_curves.png')

In [ ]:
# ── ROC curves: FF++ vs Celeb-DF ──────────────────────────────────────────────
fig, axes = plt.subplots(1, 2, figsize=(14, 6))
fig.suptitle('ROC Curves — Baseline EfficientNet-B0\n'
             'Left: FF++ Val (in-distribution) | Right: Celeb-DF (cross-dataset)',
             fontsize=13, fontweight='bold')

for ax, metrics, title, color in [
    (axes[0], ff_metrics,  f"FF++ Val\nAUC={ff_metrics['auc']:.4f}",  '#3498db'),
    (axes[1], cdf_metrics, f"Celeb-DF (cross-dataset)\nAUC={cdf_metrics['auc']:.4f}", '#e74c3c'),
]:
    fpr, tpr, _ = roc_curve(metrics['labels'], metrics['probs'])
    ax.plot(fpr, tpr, color=color, linewidth=2.5, label=f"AUC={metrics['auc']:.4f}")
    ax.plot([0, 1], [0, 1], 'k--', alpha=0.4, label='Random (0.50)')
    ax.fill_between(fpr, tpr, alpha=0.1, color=color)
    ax.set_title(title, fontweight='bold')
    ax.set_xlabel('False Positive Rate')
    ax.set_ylabel('True Positive Rate')
    ax.legend(fontsize=11)
    ax.grid(True, alpha=0.3)
    ax.set_xlim(0, 1); ax.set_ylim(0, 1)

plt.tight_layout()
plt.savefig(PLOTS_DIR / 'roc_curves.png', dpi=150, bbox_inches='tight')
plt.show()
print('✅ Saved roc_curves.png')

In [ ]:
# ── Score distributions ───────────────────────────────────────────────────────
fig, axes = plt.subplots(1, 2, figsize=(14, 5))
fig.suptitle('Prediction Score Distributions\n'
             'Well-separated = model is confident and correct',
             fontsize=13, fontweight='bold')

for ax, metrics, title in [
    (axes[0], ff_metrics,  'FF++ Validation'),
    (axes[1], cdf_metrics, 'Celeb-DF (Cross-Dataset)'),
]:
    real_scores = metrics['probs'][metrics['labels'] == 0]
    fake_scores = metrics['probs'][metrics['labels'] == 1]
    bins = np.linspace(0, 1, 40)
    ax.hist(real_scores, bins=bins, alpha=0.6, color='#2ecc71',
            label=f'Real (n={len(real_scores)})', density=True)
    ax.hist(fake_scores, bins=bins, alpha=0.6, color='#e74c3c',
            label=f'Fake (n={len(fake_scores)})', density=True)
    ax.axvline(0.5, color='black', linestyle='--', linewidth=1.5, label='Threshold=0.5')
    ax.set_xlabel('P(fake) score')
    ax.set_ylabel('Density')
    ax.set_title(f"{title}\nAUC={metrics['auc']:.4f}, Acc={metrics['acc']:.3f}",
                 fontweight='bold')
    ax.legend()
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(PLOTS_DIR / 'score_distributions.png', dpi=150, bbox_inches='tight')
plt.show()
print('✅ Saved score_distributions.png')

## Section 7 — Save Results & V8.0 Roadmap

In [ ]:
results = {
    'model':              'EfficientNet-B0 ImageNet pretrained',
    'train_on':           'FF++ Deepfakes only',
    'best_epoch':          best_epoch,
    'ff_val': {
        'auc': round(ff_metrics['auc'],  4),
        'acc': round(ff_metrics['acc'],  4),
    },
    'celeb_df': {
        'auc': round(cdf_metrics['auc'], 4),
        'acc': round(cdf_metrics['acc'], 4),
    },
    'generalization_gap': round(ff_metrics['auc'] - cdf_metrics['auc'], 4),
    'training_minutes':   round(total_time / 60, 1),
    'fixes_applied': [
        'frame pre-extraction (no VideoCapture in DataLoader)',
        'sequential read (no random seek on network FS)',
        'tqdm progress bars (training cannot silently hang)',
        'sys.stdout.flush() after every epoch row',
        'video validation step before training',
    ],
}

out_path = OUTPUT_DIR / 'baseline_results.json'
with open(out_path, 'w') as f:
    json.dump(results, f, indent=2)

print('=' * 62)
print('BASELINE COMPLETE — V8.0 ROADMAP')
print('=' * 62)
print(f"\nFF++ Val AUC  : {results['ff_val']['auc']}")
print(f"Celeb-DF AUC  : {results['celeb_df']['auc']}  ← number to beat with V8.0")
print(f"Gap           : {results['generalization_gap']}")
print(f"Training time : {results['training_minutes']} min")

print('\nV8.0 Architecture on top of this baseline:')
print('  Stage 1: EfficientNet-B0/B4 backbone (pretrained) — KEEP AS-IS')
print('  Stage 2: Per-frame embeddings → (B, T, 1280)')
print('  Stage 3: Hydra Mixer / Mamba for temporal consistency')
print('  Stage 4: Classification head')

cdf_auc = results['celeb_df']['auc']
print('\nExpected V8.0 improvement from temporal module:')
print(f'  Baseline : {cdf_auc:.4f}')
print(f'  Target   : {min(cdf_auc + 0.08, 0.97):.4f}  (+5-10% from temporal consistency)')
print(f'  SOTA     : 0.9629 (WMamba)')
print(f'\nResults saved → {out_path}')